In [49]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Image files
import os
import base64



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalshelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# username and password are here, the rest are hard coded into the CRUD file animalshelter.py
#

username = "aacuser"
password = "SNHU1234"

#kept for legacy purposes
#host = "127.0.0.1"
#port = 27017
#database = "aac"
#collection = "animals"

# create AnimalShelter oject
db = AnimalShelter(username, password)



# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned

df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# DEBUG
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Logo 
# Renamed because I have trouble with Italian/Latin/French names
#
image_filename = 'GSLogo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    # Unique identifier 
    html.Center(html.H1('Jeremy Reaban')),
    html.Div(className='buttonRow', 
            style={'display' : 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Mountain-Wilderness'),
                    html.Button(id='submit-button-three', n_clicks=0, children='Disaster'),
                    html.Button(id='submit-button-four', n_clicks=0, children='Reset')
                ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=True,
        row_selectable="single", #allow a row to be selected
        selected_rows=[0],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page

    ),
    html.Br(),
    html.Hr(),
 html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])    
])

#######################################################
# This controls both the output chart and the selection
# 
# Interaction Between Components / Controller
#######################################################
@app.callback([Output('datatable-id', "data"),Output('graph-id', "children")],
             [Input('submit-button-one', 'n_clicks'),
              Input('submit-button-two', 'n_clicks'),
              Input('submit-button-three', 'n_clicks'),
              Input('submit-button-four', 'n_clicks')
             ])
def on_click(button1, button2, button3, button4):
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    # Title of Rescue Dog Pie Chart
    rescueDog = "Default View - All Animals"
   
    if 'submit-button-one' == ctx.triggered_id:
        
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        rescueDog = "Water"
    elif 'submit-button-two' == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        rescueDog = "Mountain"
    elif 'submit-button-three' == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
        rescueDog = "Disaster"
    elif 'submit-button-four' == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({}))
        rescueDog = "Reset View - All Animals"
        
    
    
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records'), [dcc.Graph(figure = px.pie(df, names='breed', title=rescueDog))]

#This callback will highlight a cell on the data table when the user selects it

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # only return if selected_columns is not none
    if selected_columns:
       
        return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
        } for i in selected_columns]




# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]




app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:30029/
